In [100]:
a = RandomForestClassifier(max_depth=6, min_samples_leaf=3, n_estimators=32,
                         random_state=66)

In [78]:
import numpy as np
import pandas as pd
from mlens.ensemble import SuperLearner
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
from mlens.ensemble import Subsemble
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import RidgeClassifier, Perceptron, PassiveAggressiveClassifier
import optuna
from optuna.samplers import TPESampler
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.model_selection import StratifiedKFold, cross_val_score
# To see optuna progress you need to comment this row
optuna.logging.set_verbosity(optuna.logging.WARNING)

import warnings
warnings.filterwarnings('ignore')
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings(action='ignore', category=ConvergenceWarning)

In [2]:
train = pd.read_csv('../../Data/titanic_origintrain.csv')
test = pd.read_csv('../../Data/titanic_origintest.csv')

In [3]:
train['LastName'] = train['Name'].str.split(',', expand=True)[0]
test['LastName'] = test['Name'].str.split(',', expand=True)[0]
ds = pd.concat([train, test])

sur = []
died = []
for index, row in ds.iterrows():
    s = ds[(ds['LastName']==row['LastName']) & (ds['Survived']==1)]
    d = ds[(ds['LastName']==row['LastName']) & (ds['Survived']==0)]
    s=len(s)
    if row['Survived'] == 1:
        s-=1
    d=len(d)
    if row['Survived'] == 0:
        d-=1
    sur.append(s)
    died.append(d)
ds['FamilySurvived'] = sur
ds['FamilyDied'] = died

ds['FamilySize'] = ds['SibSp'] + ds['Parch'] + 1
ds['IsAlone'] = 0
ds.loc[ds['FamilySize'] == 1, 'IsAlone'] = 1
ds['Fare'] = ds['Fare'].fillna(train['Fare'].median())
ds['Embarked'] = ds['Embarked'].fillna('Q')

train = ds[ds['Survived'].notnull()]
test = ds[ds['Survived'].isnull()]
test = test.drop(['Survived'], axis=1)

train['rich_woman'] = 0
test['rich_woman'] = 0
train['men_3'] = 0
test['men_3'] = 0

train.loc[(train['Pclass']<=2) & (train['Sex']=='female'), 'rich_woman'] = 1
test.loc[(test['Pclass']<=2) & (test['Sex']=='female'), 'rich_woman'] = 1
train.loc[(train['Pclass']==3) & (train['Sex']=='male'), 'men_3'] = 1
test.loc[(test['Pclass']==3) & (test['Sex']=='male'), 'men_3'] = 1

train['rich_woman'] = train['rich_woman'].astype(np.int8)
test['rich_woman'] = test['rich_woman'].astype(np.int8)

train["Cabin"] = pd.Series([i[0] if not pd.isnull(i) else 'X' for i in train['Cabin']])
test['Cabin'] = pd.Series([i[0] if not pd.isnull(i) else 'X' for i in test['Cabin']])

train = train.drop(['PassengerId', 'Ticket', 'LastName', 'SibSp', 'Parch'], axis=1)
test = test.drop(['PassengerId', 'Ticket', 'LastName', 'SibSp', 'Parch'], axis=1)

categorical = ['Pclass', 'Sex', 'Embarked', 'Cabin']
for cat in categorical:
    train = pd.concat([train, pd.get_dummies(train[cat], prefix=cat)], axis=1)
    train = train.drop([cat], axis=1)
    test = pd.concat([test, pd.get_dummies(test[cat], prefix=cat)], axis=1)
    test = test.drop([cat], axis=1)
    
train = train.drop(['Sex_male', 'Name'], axis=1)
test =  test.drop(['Sex_male', 'Name'], axis=1)

train = train.fillna(-1)
test = test.fillna(-1)
train.head()

,Survived,Age,Fare,FamilySurvived,FamilyDied,FamilySize,IsAlone,rich_woman,men_3,Pclass_1,...,Embarked_S,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_X
0,0.0,22.0,7.2500,0,1,2,0,0,1,0,...,1,0,0,0,0,0,0,0,0,1
1,1.0,38.0,71.2833,0,0,2,0,1,0,1,...,0,0,0,1,0,0,0,0,0,0
2,1.0,26.0,7.9250,0,0,1,1,0,0,0,...,1,0,0,0,0,0,0,0,0,1
3,1.0,35.0,53.1000,0,1,2,0,1,0,1,...,1,0,0,1,0,0,0,0,0,0
4,0.0,35.0,8.0500,1,0,1,1,0,1,0,...,1,0,0,0,0,0,0,0,0,1


In [4]:
y = train['Survived']
X = train.drop(['Survived', 'Cabin_T'], axis=1)
X_test = test.copy()

X, X_val, y, y_val = train_test_split(X, y, random_state=0, test_size=0.2, shuffle=False)

In [5]:
y_test = pd.read_csv('../../Data/titanic_ytest.csv').Survived

In [7]:
sub = pd.read_csv('../../Data/submission.csv')

In [53]:
result = []

In [102]:
SEED = 44

In [104]:
class Optimizer:
    def __init__(self, metric, trials=30):
        self.metric = metric
        self.trials = trials
        self.sampler = TPESampler(seed=SEED)
        
    def objective(self, trial):
        model = create_model(trial)
        model.fit(X, y)
        preds = model.predict(X_val)
        if self.metric == 'acc':
            return accuracy_score(y_val, preds)
        else:
            return f1_score(y_val, preds)
            
    def optimize(self):
        study = optuna.create_study(direction="maximize", sampler=self.sampler)
        study.optimize(self.objective, n_trials=self.trials)
        return study.best_params

In [105]:
rf = RandomForestClassifier(random_state=SEED)
rf.fit(X, y)
preds = rf.predict(X_val)
print('Random Forest accuracy: ', accuracy_score(y_val, preds))
print('Random Forest f1-score: ', f1_score(y_val, preds))

def create_model(trial):
    max_depth = trial.suggest_int("max_depth", 2, 6)
    n_estimators = trial.suggest_int("n_estimators", 2, 150)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 10)
    model = RandomForestClassifier(min_samples_leaf=min_samples_leaf, n_estimators=n_estimators, max_depth=max_depth, random_state=666)
    return model

optimizer = Optimizer('f1')
rf_f1_params = optimizer.optimize()
rf_f1_params['random_state'] = SEED
rf_f1 = RandomForestClassifier(**rf_f1_params)
rf_f1.fit(X, y)
preds = rf_f1.predict(X_val)
print('Optimized on F1 score')
print('Optimized Random Forest: ', accuracy_score(y_val, preds))
print('Optimized Random Forest f1-score: ', f1_score(y_val, preds))

optimizer = Optimizer('acc')
rf_acc_params = optimizer.optimize()
rf_acc_params['random_state'] = SEED
rf_acc = RandomForestClassifier(**rf_acc_params)
rf_acc.fit(X, y)
preds = rf_acc.predict(X_val)
print('Optimized on accuracy')
print('Optimized Random Forest: ', accuracy_score(y_val, preds))
print('Optimized Random Forest f1-score: ', f1_score(y_val, preds))

Random Forest accuracy:  0.8491620111731844
Random Forest f1-score:  0.7804878048780488
Optimized on F1 score
Optimized Random Forest:  0.8547486033519553
Optimized Random Forest f1-score:  0.7636363636363634
Optimized on accuracy
Optimized Random Forest:  0.8547486033519553
Optimized Random Forest f1-score:  0.7592592592592593
